In [1]:
!pip install sh

  Using cached sh-1.14.1-py2.py3-none-any.whl (40 kB)


In [2]:
%run imports.ipynb
from os.path import join, isdir, isfile
import sh

imports_20150407.ipynb:26: MatplotlibDeprecationWarning: 
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  "jupyter": {


In [3]:
meta_fn = "samples.meta.txt"
dat = pd.read_csv(meta_fn, sep="\t", index_col=0)

In [4]:
dat = dat[dat.population != 'AOcol'] 
dat = dat[dat.population != 'CMgam'] 
dat = dat[dat.population != 'FRgam'] 
dat = dat[dat.population != 'GAgam'] 
dat = dat[dat.population != 'GQgam'] 
dat = dat[dat.population != 'KE'] 
dat = dat[dat.population != 'UGgam'] 
dat

,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
ox_code,,,,,,,,,,,,,,,
AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AY0087-C,Tia_dPM_46,CIcol,Cote d'Ivoire,Tiassale,Tiassale,David Weetman,David Weetman,2012,M,F,101458232,32.56,ERS311822,5.89839,-4.82293
AY0088-C,Tia_dPM_52,CIcol,Cote d'Ivoire,Tiassale,Tiassale,David Weetman,David Weetman,2012,M,F,95140584,30.75,ERS311830,5.89839,-4.82293
AY0089-C,Tia_aPM_4,CIcol,Cote d'Ivoire,Tiassale,Tiassale,David Weetman,David Weetman,2012,M,F,92383724,29.04,ERS311838,5.89839,-4.82293


In [12]:
pop_selection = metadata.population.isin({'GHcol', 'GHgam', 'BFgam', 'BFcol', 'GM', 'GW', 'GNgam', 'GNcol',
       'CIcol'}).values

In [6]:
callset_fn = callset_biallel

In [7]:
maf_min = 0.01
r2_value = 0.1
downsample_n = 100000

---------------------------------

## 3R

In [13]:
fh = callset_fn['3R']

In [14]:
from itertools import compress

In [15]:
fh_samples = [str(s) for s in callset_fn['3R']["samples"][:]]
fh_samples = list(compress(fh_samples, pop_selection))

In [16]:
len(list(compress(fh_samples, pop_selection)))

390

In [17]:
g = allel.GenotypeChunkedArray(fh['calldata/GT']).subset(sel1=pop_selection)
g

<GenotypeChunkedArray shape=(10752701, 505, 2) dtype=int8 chunks=(5251, 505, 2)
   nbytes=10.1G cbytes=472.9M cratio=21.9
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [18]:
positions = fh['variants']['POS'][:]
loci = (positions >= 1) & (positions <= 24_000_000)

In [19]:
g =g.compress(loci, axis=0)

In [20]:
pos = np.compress(loci, positions)
positions = None

In [21]:
alleles = g.count_alleles()

In [22]:
biallelic = alleles.max_allele() <= 1
g = g.compress(biallelic, axis=0)
pos = np.compress(np.array(biallelic), pos)

In [23]:
freqs = g.count_alleles().to_frequencies()
maf_ok = np.min(np.array(freqs), axis=1) >= maf_min
g = g.compress(maf_ok, axis=0)
pos = np.compress(maf_ok, pos, axis=0)

In [24]:
if pos.size > downsample_n:
        print("Downsampling... to {0} from {1}".format(downsample_n, pos.size))
        idx = np.random.choice(np.arange(0, pos.size), downsample_n, False)
        idx.sort()
        pos = np.take(pos, idx)
        g = g.take(idx, axis=0)

Downsampling... to 100000 from 865193


In [25]:
g_3R = g
g_3R

<GenotypeChunkedArray shape=(100000, 505, 2) dtype=int8 chunks=(782, 505, 2)
   nbytes=96.3M cbytes=20.3M cratio=4.8
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [28]:
h5f = h5py.File('chrom_3R_downs.h5', 'w')

In [29]:
h5f.create_dataset('3R', data=g_3R)

<HDF5 dataset "3R": shape (100000, 505, 2), type "|i1">

In [30]:
h5f.close()

In [26]:
import argparse
import h5py
import allel
import numpy as np
import gzip

In [27]:
def get_consecutive_true(a):
    if a.sum() == 0:
        return 0
    else:
        return np.diff(np.where(
            np.concatenate(([a[0]], a[:-1] != a[1:], [True])))[0])[::2].max()

chunk_size = 100000

parser = argparse.ArgumentParser(
    description='Tool to produce a vcf file from an hdf5 file')

parser.add_argument('input', help='input hdf5 file')
parser.add_argument('output', help='output file stem')

parser.add_argument('--filtermissing', '-F', action='store_false', default=True,
                    dest='keepmissing')
parser.add_argument('--cutoff', '-C', action='store', default=0.04,
                    dest='missingcutoff', type=float,
                    help='Maximum missing GTs tolerated in a sample')
parser.add_argument('--pedigree', '-P', action='store', dest='pedigree',
                    help='path to load pedigree file')

_StoreAction(option_strings=['--pedigree', '-P'], dest='pedigree', nargs=None, const=None, default=None, type=None, choices=None, help='path to load pedigree file', metavar=None)

In [31]:
# to do: add option to only filter individual crosses.
args = parser.parse_args()

with h5py.File('chrom_3R_downs.h5', mode='r') as h5_handle:
    with gzip.open(args.output + '.vcf.gz', 'wb') as f:

        f.write(b'##fileformat=VCFv4.1\n')
        f.write(b'##FORMAT=<ID=GT,Number=1,Type=String,'
                b'Description="Genotype">\n')
        f.write(b'##INFO=<ID=AC,Number=A,Type=Integer,Description="Allele count'
                b'in genotypes, for each ALT allele, in the same order as'
                b'listed">\n')

        f.write(b'##contig=<ID=2L,length=49364325>\n')
        f.write(b'##contig=<ID=2R,length=61545105>\n')
        f.write(b'##contig=<ID=3L,length=41963435>\n')
        f.write(b'##contig=<ID=3R,length=53200684>\n')
        f.write(b'##contig=<ID=UNKN,length=42389979>\n')
        f.write(b'##contig=<ID=X,length=24393108>\n')
        f.write(b'##contig=<ID=Y_unplaced,length=237045>\n')
        f.write(b'##reference=file:///data/anopheles/ag1000g/data/genome/AgamP3'
                b'/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP3.fa\n')

        reqd = [b'#CHROM', b'POS', b'ID', b'REF', b'ALT',
                b'QUAL', b'FILTER', b'INFO', b'FORMAT']

        # rememeber to act on all 1st level keys!
        # does not support multiple chromosomes currently!
        # Actually should probably add to filter script...
        assert len(h5_handle.keys()) <= 1
        for k in h5_handle.keys():

            samples = h5_handle[k]['samples'][:].tolist()
            missing_rates = np.zeros(len(samples))
            ok_samples = np.ones(len(samples), dtype="bool")

            gt = allel.GenotypeChunkedArray(
                h5_handle[k]['calldata/genotype'][:])

            if not args.keepmissing:

                missing_gt = gt.is_missing()

                for i, s in enumerate(samples):

                    consecutive_miss = get_consecutive_true(missing_gt[:, i])
                    miss_rate_i = consecutive_miss/float(missing_gt.shape[0])

                    print("Missing rate of", s, ':',
                          "{:.8f}".format(miss_rate_i),
                          "({0}/{1})".format(i+1, len(samples)))
                    missing_rates[i] = miss_rate_i

                print("Rate max:", missing_rates.max())
                ok_samples = missing_rates < args.missingcutoff

                if np.any(~ok_samples):
                    msg = "The following {0} samples are excluded as they " \
                          "have a consecutive missing gt run of >= {1} of " \
                          "all calls:".format(str(np.sum(~ok_samples)),
                                              str(args.missingcutoff))
                    print(msg)

                    for sa, rt in zip(
                            np.compress(~ok_samples, samples).tolist(),
                            np.compress(~ok_samples, missing_rates).tolist()):
                        print(sa + ": " + str(rt))

                    samples = [s.decode() for s in np.compress(
                        ok_samples, samples).tolist()]
                else:
                    print("All samples meet the missingness run threshold ({0})"
                          .format(str(args.missingcutoff)))

            if args.pedigree is not None:
                phasing.utils.create_samples_file(args.pedigree,
                                                  args.output + '.sample',
                                                  samples)

            f.write(b"\t".join(reqd + samples) + b"\n")

            number_variants = h5_handle[k]['variants']['POS'][:].size
            chunks = np.arange(0, number_variants + chunk_size, chunk_size)
            assert chunks.max() > number_variants

            for start, stop in zip(chunks[:-1], chunks[1:]):
                sl = slice(start, stop)
                positions = h5_handle[k]['variants']['POS'][sl]
                reference = h5_handle[k]['variants']['REF'][sl]
                alternate = h5_handle[k]['variants']['ALT'][sl]
                genotypes = h5_handle[k]['calldata']['genotype'][sl]
                genotypes = np.compress(ok_samples, genotypes, axis=1)
                multiple_alts = alternate.ndim > 1

                for pos, ref, alt, gt in zip(positions, reference,
                                             alternate, genotypes):
                    filterstring = 'PASS'
                    # This line filters variants where ALL genotypes are missing
                    if not args.keepmissing and np.all(gt == -1):
                        continue

                    # alt may be an np array, with several entries.
                    if multiple_alts:
                        alt = b",".join(x for x in alt if x != b'')

                    try:
                        gstr = np.apply_along_axis(b"/".join, axis=1,
                                                   arr=gt.astype("a2"))

                        genotype_str = b"\t".join([s for s in gstr]) + b"\n"
                        genotype_str = genotype_str.replace(b"-1/-1", b"./.")

                        line = b"\t".join(
                            [k.encode(), str(pos).encode()] +
                            [b'.', ref, alt, b'0', b'.', b'.', b'GT'] +
                            [genotype_str])

                        f.write(line)

                    except TypeError:
                        print(pos)
                        print(ref)
                        print(alt)
                        print(gt)
                        raise TypeError("Some data wasn't of the correct type.")

usage: ipykernel_launcher.py [-h] [--filtermissing] [--cutoff MISSINGCUTOFF]
                             [--pedigree PEDIGREE]
                             input output
ipykernel_launcher.py: error: the following arguments are required: output


SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import argparse
import h5py
import allel
import numpy as np
import gzip


def get_consecutive_true(a):
    if a.sum() == 0:
        return 0
    else:
        return np.diff(np.where(
            np.concatenate(([a[0]], a[:-1] != a[1:], [True])))[0])[::2].max()

chunk_size = 200000

parser = argparse.ArgumentParser(
    description='Tool to produce a vcf file from an hdf5 file')

parser.add_argument('input', help='input hdf5 file')
parser.add_argument('output', help='output file stem')

parser.add_argument('--filtermissing', '-F', action='store_false', default=True,
                    dest='keepmissing')
parser.add_argument('--cutoff', '-C', action='store', default=0.04,
                    dest='missingcutoff', type=float,
                    help='Maximum missing GTs tolerated in a sample')
parser.add_argument('--pedigree', '-P', action='store', dest='pedigree',
                    help='path to load pedigree file')

# to do: add option to only filter individual crosses.
args = parser.parse_args()

with h5py.File(args.input, mode='r') as h5_handle:
    with gzip.open(args.output + '.vcf.gz', 'wb') as f:

        f.write(b'##fileformat=VCFv4.1\n')
        f.write(b'##FORMAT=<ID=GT,Number=1,Type=String,'
                b'Description="Genotype">\n')
        f.write(b'##INFO=<ID=AC,Number=A,Type=Integer,Description="Allele count'
                b'in genotypes, for each ALT allele, in the same order as'
                b'listed">\n')

        f.write(b'##contig=<ID=2L,length=49364325>\n')
        f.write(b'##contig=<ID=2R,length=61545105>\n')
        f.write(b'##contig=<ID=3L,length=41963435>\n')
        f.write(b'##contig=<ID=3R,length=53200684>\n')
        f.write(b'##contig=<ID=UNKN,length=42389979>\n')
        f.write(b'##contig=<ID=X,length=24393108>\n')
        f.write(b'##contig=<ID=Y_unplaced,length=237045>\n')
        f.write(b'##reference=file:///data/anopheles/ag1000g/data/genome/AgamP3'
                b'/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP3.fa\n')

        reqd = [b'#CHROM', b'POS', b'ID', b'REF', b'ALT',
                b'QUAL', b'FILTER', b'INFO', b'FORMAT']

        # rememeber to act on all 1st level keys!
        # does not support multiple chromosomes currently!
        # Actually should probably add to filter script...
        assert len(h5_handle.keys()) <= 1
        for k in h5_handle.keys():

            samples = h5_handle[k]['samples'][:].tolist()
            missing_rates = np.zeros(len(samples))
            ok_samples = np.ones(len(samples), dtype="bool")

            gt = allel.GenotypeChunkedArray(
                h5_handle[k]['calldata/genotype'][:])

            if not args.keepmissing:

                missing_gt = gt.is_missing()

                for i, s in enumerate(samples):

                    consecutive_miss = get_consecutive_true(missing_gt[:, i])
                    miss_rate_i = consecutive_miss/float(missing_gt.shape[0])

                    print("Missing rate of", s, ':',
                          "{:.8f}".format(miss_rate_i),
                          "({0}/{1})".format(i+1, len(samples)))
                    missing_rates[i] = miss_rate_i

                print("Rate max:", missing_rates.max())
                ok_samples = missing_rates < args.missingcutoff

                if np.any(~ok_samples):
                    msg = "The following {0} samples are excluded as they " \
                          "have a consecutive missing gt run of >= {1} of " \
                          "all calls:".format(str(np.sum(~ok_samples)),
                                              str(args.missingcutoff))
                    print(msg)

                    for sa, rt in zip(
                            np.compress(~ok_samples, samples).tolist(),
                            np.compress(~ok_samples, missing_rates).tolist()):
                        print(sa + ": " + str(rt))

                    samples = [s.decode() for s in np.compress(
                        ok_samples, samples).tolist()]
                else:
                    print("All samples meet the missingness run threshold ({0})"
                          .format(str(args.missingcutoff)))

            if args.pedigree is not None:
                phasing.utils.create_samples_file(args.pedigree,
                                                  args.output + '.sample',
                                                  samples)

            f.write(b"\t".join(reqd + samples) + b"\n")

            number_variants = h5_handle[k]['variants']['POS'][:].size
            chunks = np.arange(0, number_variants + chunk_size, chunk_size)
            assert chunks.max() > number_variants

            for start, stop in zip(chunks[:-1], chunks[1:]):
                sl = slice(start, stop)
                positions = h5_handle[k]['variants']['POS'][sl]
                reference = h5_handle[k]['variants']['REF'][sl]
                alternate = h5_handle[k]['variants']['ALT'][sl]
                genotypes = h5_handle[k]['calldata']['genotype'][sl]
                genotypes = np.compress(ok_samples, genotypes, axis=1)
                multiple_alts = alternate.ndim > 1

                for pos, ref, alt, gt in zip(positions, reference,
                                             alternate, genotypes):
                    filterstring = 'PASS'
                    # This line filters variants where ALL genotypes are missing
                    if not args.keepmissing and np.all(gt == -1):
                        continue

                    # alt may be an np array, with several entries.
                    if multiple_alts:
                        alt = b",".join(x for x in alt if x != b'')

                    try:
                        gstr = np.apply_along_axis(b"/".join, axis=1,
                                                   arr=gt.astype("a2"))

                        genotype_str = b"\t".join([s for s in gstr]) + b"\n"
                        genotype_str = genotype_str.replace(b"-1/-1", b"./.")

                        line = b"\t".join(
                            [k.encode(), str(pos).encode()] +
                            [b'.', ref, alt, b'0', b'.', b'.', b'GT'] +
                            [genotype_str])

                        f.write(line)

                    except TypeError:
                        print(pos)
                        print(ref)
                        print(alt)
                        print(gt)
                        raise TypeError("Some data wasn't of the correct type.")